In [1]:
from pynq.overlays.base import BaseOverlay
import pynq.lib.rgbled as rgbled

import threading
import time
import multiprocessing
import os

base = BaseOverlay("base.bit")

In [2]:
def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: number of times to blink the LED
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 4)
    '''
    if n < 4:
        led = base.leds[n]
        for i in range(int(t)):
            led.toggle()
            time.sleep(d)

    else:
        led = rgbled.RGBLED(4)
        # RGB LEDs do not have "toggle" API
        for i in range(int(t / 2)):
            led.off()
            time.sleep(d)
            led.on(0x2)     # Green is 2'b010
            time.sleep(d)

    led.off()

def led_off(n):
    if n < 4:
        led = base.leds[n]
    else:
        led = rgbled.RGBLED(4)

    led.off()

def worker_t(forks, num, btn_press_event, t_eat= 1, t_nap = 2, blocking = False):
    # Refer to the table figure in report
    # Philosopher #       Left fork      Right fork
    # Philosopher 0 needs fork 0    and  4
    # Philosopher 1 needs fork 1    and  0
    # Philosopher 2 needs fork 2    and  1
    # Philosopher 3 needs fork 3    and  2
    # Philosopher 4 needs fork 4    and  3

    l_fork_num = num
    if num != 0:
        r_fork_num = num - 1
    else:
        r_fork_num = 4

    # Blink the LED faster while eating
    d_eat = 0.01
    d_nap = 0.2

    if not blocking:
        while not btn_press_event.is_set():
            led_off(num)
            l_fork_acquired = False
            r_fork_acquired = False
            starving_time_start = time.time()
            
            # Starve the philosopher until they aqcuire both the
            # left and right forks
            while not (l_fork_acquired and r_fork_acquired):
                time.sleep(0.01)

                # Kill the thread if button is pressed
                if btn_press_event.is_set():
                    return

                # Try to acquire both left & right forks
                l_fork_acquired = forks[l_fork_num].acquire(False)
                r_fork_acquired = forks[r_fork_num].acquire(False)

                # If both were not acquired, then release if only one
                # was acquired
                # Print starving only once to keep the prints
                if not (l_fork_acquired and r_fork_acquired):
                    if l_fork_acquired:
                        forks[l_fork_num].release()
                    elif r_fork_acquired:
                        forks[r_fork_num].release()

            print("\nPhilosopher %d starved for %.2f second(s)" % (num, time.time() - starving_time_start))
            print(f"\nPhilosopher {num} is eating")
            blink(t_eat / d_eat, d_eat, num)
            forks[l_fork_num].release()
            forks[r_fork_num].release()
            print(f"\nPhilosopher {num} is napping")
            blink(t_nap / d_nap, d_nap, num)
            time.sleep(0) # yield

    else:
        while not btn_press_event.is_set():
            led_off(num)
            l_fork_acquired = forks[l_fork_num].acquire(True)
            r_fork_acquired = forks[r_fork_num].acquire(True)
            time.sleep(0.01)

            print(f"\nPhilosopher {num} is eating")
            blink(t_eat / d_eat, d_eat, num)
            forks[l_fork_num].release()
            forks[r_fork_num].release()
            print(f"\nPhilosopher {num} is napping")
            blink(t_nap / d_nap, d_nap, num)
            time.sleep(0) # yield

In [5]:
# Define the 5 forks
forks = []
for _ in range(5):
    forks.append(threading.Lock())

# Create a button press event to kill the thread
btn_press_event = multiprocessing.Event()
btn_press_event.clear()

def p_spawn_philosopher_threads(_cpu, forks, btn_press_event):
    # Initialize and launch the philosopher threads
    threads = []
    for i in range(5):
        t = threading.Thread(target=worker_t, args=(forks, i, btn_press_event, 1, 2, True))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

def p_btn_press_check(_cpu, btn_press_event):
    global base
    while True:
        time.sleep(0.01)
        if base.btns_gpio.read():
            btn_press_event.set()
            print("\nButton pressed!")
            break

# Launch process1 on CPU0
p1 = multiprocessing.Process(
         target=p_spawn_philosopher_threads,
         args=(0, forks, btn_press_event))

# Start the process before issuing the taskset OS
# command to bind the CPU affinity. PID is not
# populated until process is started.
p1.start()

# -p is a bit mask (without the "-c" option). CPU0 would be 0x1.
print("\n")
os.system("taskset -p {} {}".format(0x1, p1.pid))

# Launch process2 on CPU1
p2 = multiprocessing.Process(
         target=p_btn_press_check, args=(1, btn_press_event))

# Start the process before issuing the taskset OS
# command to bind the CPU affinity. PID is not
# populated until process is started.
p2.start()

# -p is a bit mask (without the "-c" option), CPU1 would be 2'b10.
print("\n")
os.system("taskset -p {} {}".format(0x2, p2.pid))

p2.join() # wait for process2 to finish
p1.join() # wait for process1 to finish



pid 14853's current affinity mask: 3
pid 14853's new affinity mask: 1

Philosopher 0 is eating


pid 14865's current affinity mask: 3
pid 14865's new affinity mask: 2

Philosopher 0 is napping

Philosopher 1 is eating

Philosopher 1 is napping

Philosopher 2 is eating

Philosopher 2 is napping

Philosopher 3 is eating

Philosopher 3 is napping

Philosopher 4 is eating

Philosopher 4 is napping
Philosopher 0 is eating


Philosopher 0 is napping

Philosopher 1 is eating

Philosopher 1 is napping

Philosopher 2 is eating

Philosopher 2 is napping

Philosopher 3 is eating

Philosopher 3 is napping

Philosopher 4 is eating

Philosopher 4 is napping

Philosopher 0 is eating

Philosopher 0 is napping

Philosopher 1 is eating

Philosopher 1 is napping

Philosopher 2 is eating

Philosopher 2 is napping

Philosopher 3 is eating

Philosopher 3 is napping

Philosopher 4 is eating

Philosopher 4 is napping

Philosopher 0 is eating

Philosopher 0 is napping

Philosopher 1 is eating

Philosopher 1 

In [4]:
import random

t_eat_min = 1
t_eat_max = 5

t_nap_min = 6
t_nap_max = 10

t_eat = random.randint(t_eat_min, t_eat_max)
t_nap = random.randint(t_nap_min, t_nap_max)

# Define the 5 forks
forks = []
for _ in range(5):
    forks.append(threading.Lock())

# Create a button press event to kill the thread
btn_press_event = multiprocessing.Event()
btn_press_event.clear()

def p_spawn_philosopher_threads(_cpu, forks, btn_press_event):
    # Initialize and launch the philosopher threads
    threads = []
    for i in range(5):
        t = threading.Thread(target=worker_t, args=(forks, i, btn_press_event, t_eat, t_nap))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

def p_btn_press_check(_cpu, btn_press_event):
    global base
    while True:
        time.sleep(0.01)
        if base.btns_gpio.read():
            btn_press_event.set()
            print("\nButton pressed!")
            break

# Launch process1 on CPU0
p1 = multiprocessing.Process(
         target=p_spawn_philosopher_threads,
         args=(0, forks, btn_press_event))

# Start the process before issuing the taskset OS
# command to bind the CPU affinity. PID is not
# populated until process is started.
p1.start()

# -p is a bit mask (without the "-c" option). CPU0 would be 0x1.
print("\n")
os.system("taskset -p {} {}".format(0x1, p1.pid))

# Launch process2 on CPU1
p2 = multiprocessing.Process(
         target=p_btn_press_check, args=(1, btn_press_event))

# Start the process before issuing the taskset OS
# command to bind the CPU affinity. PID is not
# populated until process is started.
p2.start()

# -p is a bit mask (without the "-c" option), CPU1 would be 2'b10.
print("\n")
os.system("taskset -p {} {}".format(0x2, p2.pid))

p2.join() # wait for process2 to finish
p1.join() # wait for process1 to finish



pid 13565's current affinity mask: 3
pid 13565's new affinity mask: 1

Philosopher 3 starved for 0.02 second(s)
Philosopher 0 starved for 0.01 second(s)


Philosopher 3 is eating

Philosopher 0 is eating


pid 13577's current affinity mask: 3
pid 13577's new affinity mask: 2

Philosopher 3 is napping
Philosopher 2 starved for 5.24 second(s)


Philosopher 0 is napping
Philosopher 4 starved for 5.27 second(s)


Philosopher 2 is eating
Philosopher 4 is eating


Philosopher 2 is napping
Philosopher 1 starved for 10.46 second(s)


Philosopher 4 is napping
Philosopher 1 is eating


Philosopher 3 starved for 0.01 second(s)

Philosopher 3 is eating

Philosopher 1 is napping
Philosopher 0 starved for 2.31 second(s)


Philosopher 0 is eating

Philosopher 3 is napping

Philosopher 2 starved for 0.01 second(s)

Philosopher 2 is eating

Philosopher 0 is napping
Philosopher 4 starved for 2.25 second(s)


Philosopher 4 is eating

Button pressed!

Philosopher 2 is napping

Philosopher 4 is napping


In [ ]:
import asyncio

# State of the philosophers is encoded as follows
# 0 - Eating
# 1 - Napping
# 2 - Starving
class Philosopher:
    def __init__(self, philo_id, init_state = 2):
        global base
        self.philo_id = philo_id
        self.state = init_state
        if philo_id < 4:
            self.led = base.leds[philo_id]
        elif philo_id == 4:
            self.led = rgbled.RGBLED(4)
        print(f"Philosopher {self.philo_id} created.")
        
        # Start a workloop
        self.loop = asyncio.new_event_loop()
        self.loop.create_task(self.update_state())
        self.loop.create_task(self._blink())
        self.loop.run_forever()

    def eat(self):
        self.state = 0

    def nap(self):
        self.state = 1

    def starve(self):
        self.state = 2

    async def update_state(self, new_state = 2):
        while True:
            if self.state != new_state:
                print(f"Philosopher {self.philo_id} is going to state {self.state}.")
                self.state = new_state

                # When a philosopher is waiting for forks, its
                # LED should be off to indicate “starving”.
                if self.state == 2:
                    self.led.off()

                # When a philosopher is napping, the LED should
                # blink at a lower rate to indicate “napping”.
                elif self.state == 1:
                    self.blink_duration = 1

                # When one of the philosophers is eating, both
                # forks are used by that philosopher, and the
                # LED should blink at a higher rate to indicate
                # “eating”.
                elif self.state == 0:
                    self.blink_duration = 0.5

            await asyncio.sleep(0.01)

    async def _blink(self):
        '''
        Function to blink the LEDs
        Params:
          d: duration (in seconds) for the LED to be on/off
        '''
        while True:
            if self.state != 2:
                await asyncio.sleep(self.blink_duration)
                self.led.toggle()

In [ ]:
# Define the 5 forks
forks = []
for _ in range(5):
    forks.append(threading.Lock())

# Create a button press event to kill the thread
btn_press_event = threading.Event()
btn_press_event.clear()

def btn_press_t(btn_press_event):
    global base
    print("\nbtn_press_t spawned")
    while True:
        time.sleep(0.01)
        if base.btns_gpio.read():
            print("\nButton pressed!")
            btn_press_event.set()
            break

# Initialize and launch the philosopher threads and
# one thread to monitor the button press.
threads = []
for i in range(5):
    t = threading.Thread(target=worker_t, args=(forks, i, btn_press_event))
    t.daemon = True
    threads.append(t)

t = threading.Thread(target=btn_press_t, args=(btn_press_event, ))
t.daemon = True
threads.append(t)

for t in threads:
    t.start()